<a href="https://colab.research.google.com/github/tatsuhiko-suyama/Something-/blob/main/4_28_alpha_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
alpha_sweep_K4M4.py  –  Robust MVP α-sweep  (K = M = 4)
2025-04-28  patched version  (Q1–Q5 反映)

変更ログ
--------
* Q1  射影勾配を用いた KKT 停止判定
* Q2  BB ステップ幅 γ_BB の符号チェック（γ_BB > 0 のみ採用）
* Q3  `seed` 引数で乱初期点を許可（未指定なら一様重み）
* Q4  dpi* 計算を可変幅 h で中央差分（端点は片側差分）
* Q5  λ ベクトルは **0 埋め**（欠測は 0.0）に統一
"""

# ---------------------------------------------------------------------
# 0. IMPORTS & GLOBAL CONSTANTS
# ---------------------------------------------------------------------
import numpy as np
import pandas as pd
import itertools, logging, sys
from datetime import datetime

K, M         = 4, 4
mu_tilde     = 0.03
ALPHA_MIN    = 0.01
ALPHA_MAX    = 2.0
N_ALPHA      = 101                      # ← 必要なら 201 へ拡張可
ALPHA_GRID   = np.linspace(ALPHA_MIN, ALPHA_MAX, N_ALPHA)

MAX_OPT_ITERS            = 1000
tol_grad                 = 1e-8
step_size_norm_threshold = 1e-12        # tighten
lr_init, lr_max          = 0.15, 1.0

FINITE_DIFF_EPS          = 1e-12        # 安定化用

# ---------------------------------------------------------------------
# 1. SYNTHETIC DATA
# ---------------------------------------------------------------------
r_base = np.array([0.02, 0.03, 0.04, 0.05])
Delta_A = np.array([[+1, -1, +1, +1],
                    [+2, -2, -2, -2],
                    [-2, +3, +3, -3],
                    [+4, +4, -4, +4]], dtype=float)
Delta_r_m     = Delta_A / 1000
sigma_base    = np.array([0.20, 0.25, 0.30, 0.35])
Delta_sigma_m = Delta_A / 100

def block_corr(rho_in: float, rho_out: float) -> np.ndarray:
    C = np.full((K, K), rho_out)
    np.fill_diagonal(C, 1.0)
    sectA, sectB = [0, 1], [2, 3]
    for i in sectA:
        for j in sectA:
            if i != j:
                C[i, j] = rho_in
    for i in sectB:
        for j in sectB:
            if i != j:
                C[i, j] = rho_in
    return C

C_base = block_corr(0.75, 0.50)
tilde_C_list = [
    block_corr(0.85, 0.40),
    block_corr(0.65, 0.40),
    block_corr(0.65, 0.60),
    block_corr(0.85, 0.60),
]
beta_corr = 0.5

def mix_corr(C0, C1, alpha, beta=0.5, eps=1e-4):
    C = (1 - beta * alpha) * C0 + beta * alpha * C1
    eigval, eigvec = np.linalg.eigh(C)
    eigval_clip = np.clip(eigval, eps, None)
    C_spd = eigvec @ np.diag(eigval_clip) @ eigvec.T
    D = np.sqrt(np.diag(C_spd))
    return C_spd / np.outer(D, D), float(eigval_clip.min())

# ---------------------------------------------------------------------
# 2. SIMPLEX PROJECTION
# ---------------------------------------------------------------------
def proj_simplex(v: np.ndarray) -> np.ndarray:
    v = np.asarray(v, float)
    if (v >= 0).all() and np.isclose(v.sum(), 1.0, atol=1e-9):
        return v
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - 1
    rho = np.where(u - cssv / (np.arange(len(u)) + 1) > 0)[0][-1]
    theta = cssv[rho] / (rho + 1)
    return np.maximum(v - theta, 0.0)

# ---------------------------------------------------------------------
# 3. PARAMETER CONSTRUCTION
# ---------------------------------------------------------------------
def build_params(alpha: float):
    R_th = r_base[:, None] + alpha * Delta_r_m
    Sigma_th = np.zeros((K, K, M))
    lambda_log = []
    for m in range(M):
        sig_vec = sigma_base + alpha * Delta_sigma_m[:, m]
        diag_sig = np.diag(sig_vec)
        C_alpha, lam_min = mix_corr(C_base, tilde_C_list[m], alpha,
                                    beta=beta_corr)
        lambda_log.append(lam_min)
        Sigma_th[:, :, m] = diag_sig @ C_alpha @ diag_sig + np.outer(R_th[:, m], R_th[:, m])
    return R_th, Sigma_th, lambda_log

# ---------------------------------------------------------------------
# 4. INNER MINIMISATION  (exact solve over active subsets)
# ---------------------------------------------------------------------
def inner_opt(w, R, Sigma,
              tol_feas   = 1e-9,
              lam_tol    = -1e-13,
              rank_scale = 1e-10):
    rank_tol = rank_scale * np.linalg.norm(R)
    Ex = R @ w
    Vw = sum(w[m] * Sigma[:, :, m] for m in range(M)) - np.outer(Ex, Ex)
    try:
        Vinv = np.linalg.inv(Vw)
    except np.linalg.LinAlgError:
        return None, np.inf, Ex, None, None

    best_val = np.inf
    best_pi = best_lam = best_subset = None

    for s in range(1, M + 1):
        for subset in itertools.combinations(range(M), s):
            A = R[:, subset].T
            if np.linalg.matrix_rank(A, tol=rank_tol) < s:
                continue
            Mmat = A @ Vinv @ A.T
            if np.linalg.matrix_rank(Mmat, tol=rank_tol) < s:
                continue
            lam = np.linalg.solve(Mmat, mu_tilde * np.ones(s))
            if np.any(lam < lam_tol):
                continue
            pi = Vinv @ A.T @ lam
            if np.any(R.T @ pi < mu_tilde - tol_feas):
                continue
            val = pi @ Vw @ pi
            if val + 1e-12 < best_val:
                best_val, best_pi, best_lam, best_subset = val, pi, lam, subset

    if best_pi is None:
        return None, np.inf, Ex, None, None
    return best_pi, best_val, Ex, best_lam, best_subset

# ---------------------------------------------------------------------
# 5. OUTER OBJECTIVE & GRADIENT
# ---------------------------------------------------------------------
def H_and_grad(w, R, Sigma):
    pi, val, Ex, lam, subset = inner_opt(w, R, Sigma)
    if val == np.inf:
        return val, None, pi, lam, subset
    grad = np.array([
        pi @ Sigma[:, :, m] @ pi - 2 * (pi @ R[:, m]) * (pi @ Ex)
        for m in range(M)
    ])
    return val, grad, pi, lam, subset

# ---------------------------------------------------------------------
# 6. OUTER OPTIMISER  (PG + Barzilai–Borwein + Armijo back-tracking)
# ---------------------------------------------------------------------
def optimise_outer(R, Sigma, *, start=None, seed=None):
    if start is None:
        if seed is None:
            w = np.ones(M) / M               # default uniform
        else:
            rng = np.random.default_rng(seed)
            w = rng.random(M)
            w = proj_simplex(w)
    else:
        w = proj_simplex(start)

    val, g, pi, lam, subset = H_and_grad(w, R, Sigma)
    best = (val, w.copy(), pi, lam, subset)
    lr   = lr_init
    g_prev, w_prev = g.copy(), w.copy()

    for it in range(1, MAX_OPT_ITERS + 1):

        # --- KKT 残差（射影勾配）
        proj_grad = w - proj_simplex(w - g)
        if np.linalg.norm(proj_grad) < tol_grad:
            break

        # --- BB ステップ幅
        if it > 1:
            dw, dg = w - w_prev, g - g_prev
            denom = np.dot(dg, dg)
            if denom > 1e-12:
                gamma_bb = np.dot(dw, dg) / denom
                if gamma_bb > 0.0:                     # ascend only
                    lr = np.clip(gamma_bb, 1e-4, lr_max)

        # --- Armijo line-search
        lr_curr = lr
        accept  = False
        while lr_curr >= 1e-6:
            w_trial = proj_simplex(w + lr_curr * g)    # ascent step
            if np.linalg.norm(w_trial - w) < step_size_norm_threshold:
                lr_curr = 0.0
                break
            val_t, g_t, pi_t, lam_t, subset_t = H_and_grad(w_trial, R, Sigma)
            if val_t >= val - 1e-12:                   # ascent condition
                w_prev, g_prev = w.copy(), g.copy()
                w, val, g = w_trial, val_t, g_t
                pi, lam, subset = pi_t, lam_t, subset_t
                if val > best[0]:
                    best = (val, w.copy(), pi, lam, subset)
                lr = min(lr_curr * 1.2, lr_max)
                accept = True
                break
            lr_curr *= 0.5
        if not accept:
            break

    return (*best, it)

# ---------------------------------------------------------------------
# 7. FINITE DIFFERENCE HELPERS
# ---------------------------------------------------------------------
def finite_diff(f_p, f_m, h):
    if f_p is None or f_m is None:
        return np.nan
    return (f_p - f_m) / max(h * 2, FINITE_DIFF_EPS)

def finite_diff_vec(v_p, v_m, h, length):
    if v_p is None or v_m is None:
        return np.full(length, np.nan)
    return (v_p - v_m) / max(h * 2, FINITE_DIFF_EPS)

# ---------------------------------------------------------------------
# 8. α-SWEEP
# ---------------------------------------------------------------------
def robust_opt(alpha, *, seed=None):
    R_th, Sigma_th, _ = build_params(alpha)
    H_star, w_star, pi_star, lam_star, subset_star, iters = optimise_outer(
        R_th, Sigma_th, seed=seed)
    return dict(H_star=H_star, w_star=w_star, pi_star=pi_star,
                lam_star=lam_star, subset=subset_star, R=R_th, iters=iters)

def run_alpha_sweep(csv_path="alpha_sweep_K4M4.csv", *, seed_outer=None):
    log = logging.getLogger("sweep")
    log.setLevel(logging.INFO)
    log.addHandler(logging.StreamHandler(sys.stdout))

    col_w   = [f"w*_m{m}"   for m in range(M)]
    col_pi  = [f"pi*_k{k}"  for k in range(K)]
    col_ret = [f"cstr_ret_m{m}" for m in range(M)]
    col_lam = [f"lam_m{m}"  for m in range(M)]
    col_dpi = [f"dpi*_k{k}" for k in range(K)]
    columns = (["alpha", "H_star", "supp_w", "lambda_min", "iterations"]
               + col_w + col_pi + col_ret + col_lam + col_dpi)

    rows, cache = [], {}
    t0 = datetime.now()
    log.info("========== α-Sweep START ==========")

    for idx, alpha in enumerate(ALPHA_GRID):
        log.info("α = %.5f  (%d/%d)", alpha, idx + 1, N_ALPHA)

        # -- utility with memoisation ---------------------------------
        def get(a):
            if a is None:
                return None
            if a not in cache:
                cache[a] = robust_opt(a, seed=seed_outer)
            return cache[a]

        res_c   = get(alpha)
        H_star  = res_c["H_star"]
        w_star  = res_c["w_star"]
        pi_star = res_c["pi_star"]
        lam_star= res_c["lam_star"]
        subset  = res_c["subset"]
        R_th    = res_c["R"]
        iters   = res_c["iters"]

        # SPD λ_min (smallest eigen-value) logging
        _, _, lambda_log = build_params(alpha)
        lambda_min_val   = float(min(lambda_log))

        # --- central / one-sided finite differences ------------------
        prev_alpha = ALPHA_GRID[idx - 1] if idx > 0 else None
        next_alpha = ALPHA_GRID[idx + 1] if idx < N_ALPHA - 1 else None
        h_left  = alpha - prev_alpha if prev_alpha is not None else None
        h_right = next_alpha - alpha if next_alpha is not None else None

        if prev_alpha is not None and next_alpha is not None:
            h = min(h_left, h_right)
            H_p  = get(alpha + h)["H_star"]
            H_m  = get(alpha - h)["H_star"]
            pi_p = get(alpha + h)["pi_star"]
            pi_m = get(alpha - h)["pi_star"]
        elif prev_alpha is None:           # left boundary → forward diff
            h = h_right
            H_p  = get(next_alpha)["H_star"]
            H_m  = H_star
            pi_p = get(next_alpha)["pi_star"]
            pi_m = pi_star
        else:                              # right boundary → backward diff
            h = h_left
            H_p  = H_star
            H_m  = get(prev_alpha)["H_star"]
            pi_p = pi_star
            pi_m = get(prev_alpha)["pi_star"]

        dpi_star = finite_diff_vec(pi_p, pi_m, h, K)

        # --- constraint returns and λ vector -------------------------
        cstr_ret = R_th.T @ pi_star if pi_star is not None else np.full(M, np.nan)
        lam_vec  = np.zeros(M)                                # ← Q5: 0 埋め
        if lam_star is not None and subset is not None:
            for j, m_idx in enumerate(subset):
                lam_vec[m_idx] = lam_star[j]

        row = ([alpha, H_star,
                int((w_star > 1e-6).sum()) if w_star is not None else np.nan,
                lambda_min_val, iters]
               + list(w_star) + list(pi_star) + list(cstr_ret) + list(lam_vec) + list(dpi_star))
        rows.append(row)

    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(csv_path, index=False, float_format="%.10g")
    log.info("CSV written to %s  (elapsed %.1fs)", csv_path,
             (datetime.now() - t0).total_seconds())
    return df

# ---------------------------------------------------------------------
# 9. MAIN
# ---------------------------------------------------------------------
if __name__ == "__main__":
    run_alpha_sweep()

========== α-Sweep START ==========


INFO:sweep:========== α-Sweep START ==========


α = 0.01000  (1/101)


INFO:sweep:α = 0.01000  (1/101)


α = 0.02990  (2/101)


...

INFO:sweep:α = 0.10950  (6/101)


α = 0.12940  (7/101)


INFO:sweep:α = 0.12940  (7/101)


α = 0.14930  (8/101)


INFO:sweep:α = 0.14930  (8/101)
